Rebuilding Steps, Example Network, Integrated in SNMan 'Multi' Rebuild
====================
xxx

In [11]:
import warnings
warnings.filterwarnings('ignore')

import copy
import os
import sys
import networkx as nx
import numpy as np
import shutil

if 0:
    # Makes a local copy of current snman code
    source_dir = os.path.join('C:',os.sep,'Users','lballo','polybox','GIT','github','snman','snman')
    destination_dir = os.path.join('..','snman')
    shutil.copytree(source_dir, destination_dir, dirs_exist_ok=True)

import snman
from snman.constants import *
from snman import osmnx_customized as oxc
from snman import *

#PERIMETER = '_paper2_example'
PERIMETER = '_debug'
#PERIMETER = 'zrh_north-west'
#PERIMETER = 'ebc_zrh_v01_10km'

data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
export_path = os.path.join(data_directory, 'outputs', '_debug')

CRS_internal = 2056      # for Zurich

Loading data
------------
Loads the prepared simplified street graph, as well as other datasets such as perimeters, rebuilding regions and measurement regions

In [12]:
print('Load street graph')
G = snman.io.load_street_graph(
    os.path.join(process_path, 'street_graph_edges.gpkg'),
    os.path.join(process_path, 'street_graph_nodes.gpkg'),
    crs=CRS_internal
)

street_graph.organize_edge_directions(G)

Load street graph


In [13]:
print('Load rebuilding regions')
# Polygons that define which streets will be reorganized
rebuilding_regions_gdf = snman.io.load_rebuilding_regions(
    #os.path.join(process_path, 'rebuilding_regions.gpkg'),
    os.path.join(inputs_path, 'rebuilding_regions', 'rebuilding_regions.gpkg'),
    #only_active=True,
    #filter_ids=['zrh-4'],
    #filter_ids=['zrh-19'],
    #filter_ids=['zrh-main'],
    #filter_ids=['playground-1'],
    filter_ids=['paper2_example'],
    crs=CRS_internal
)

print('Load access needs')
access_needs = snman.io.load_access_needs(
    os.path.join(inputs_path, 'switzerland', 'zurich', 'statpop', 'statpop17_zurich_city_agg_on_buildings.gpkg'),
    CRS_internal
)

access_needs['parking_spots_needed'] = access_needs['residents'] / 60

Load rebuilding regions
Load access needs


In [14]:
import geopandas as gpd
from snman import *
from snman.rebuilding import *

HLAs = multi_rebuild_regions(
    G, rebuilding_regions_gdf, access_needs,
    save_steps_path=os.path.join(export_path, 'steps'),
    gravity_iterations=10
)
HLAs = list(filter(lambda HLA: HLA is not None, HLAs))

truncating street graph
done truncating
GIVEN LANES
REMOVE PARKING
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
REMOVE CAR LANES
iteration 1
removed (153, 152, '152-153-0-0')
iteration 2
removed (152, 153, '152-153-0-0')
iteration 3
removed (143, 301, '143-301-0-0')
iteration 4
removed (425, 143, '143-425-0-0')
iteration 5
removed (511, 425, '425-511-0-0')
iteration 6
removed (301, 143, '143-301-0-0')
iteration 7
fixed [143, 425, '143-425-0-0-1']
iteration 8
removed (425, 511, '425-511-0-0')
iteration 9
removed (154, 346, '154-346-0-0')
iteration 10
removed (346, 154, '154-346-0-0')
iteration 11
removed (143, 157, '143-157-0-0')
iteration 12
removed (157, 425, '157-425-0-2')
iteration 13
removed (301, 153, '153-301-0-0')
iteration 14
removed (153, 150, '150-153-0-2')
iteration 15
removed (154, 152, '152-154-0-0')
iteration 16
removed (150, 154, '1

In [15]:
import networkx as nx

Hs,Ls,As = zip(*HLAs)
A = nx.compose_all(As)
L = nx.compose_all(Ls)

#export_step('_debug', L=L, A=A)

In [16]:
print('Reorder lanes')
snman.space_allocation.reorder_lanes(G, lanes_attribute=KEY_LANES_DESCRIPTION)
#snman.space_allocation.reorder_lanes(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)

print('Normalize cycling infrastructure')
snman.space_allocation.normalize_cycling_lanes(G, lanes_key=KEY_LANES_DESCRIPTION)
#snman.space_allocation.normalize_cycling_lanes(G, lanes_key=KEY_LANES_DESCRIPTION_AFTER)

snman.street_graph.organize_edge_directions(G)

Reorder lanes
Normalize cycling infrastructure


In [17]:
print('Add elevation')
G = oxc.elevation.add_node_elevations_raster(
    G,
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'ch_dhm_25', 'ch_dhm_2056.tif'),
    raster_crs=2056,
    #os.path.join(inputs_path, 'copernicus_dem', 'Copernicus_DSM_10_N41_00_W088_00', 'DEM', 'Copernicus_DSM_10_N41_00_W088_00_DEM.tif'),
    #raster_crs=4326, #Chicago
    graph_crs=CRS_internal,
    cpus=1
)
G = oxc.elevation.add_edge_grades(G, add_absolute=False)

Add elevation
                  x             y
osmid                            
0      2.679349e+06  1.253328e+06
1      2.679318e+06  1.253339e+06
2      2.681238e+06  1.253939e+06
3      2.682650e+06  1.250982e+06
4      2.680482e+06  1.251586e+06
...             ...           ...
767    2.682760e+06  1.250814e+06
768    2.682739e+06  1.250846e+06
769    2.682727e+06  1.250864e+06
770    2.682715e+06  1.250881e+06
771    2.683563e+06  1.252448e+06

[719 rows x 2 columns]


In [18]:
if 0:

    L = snman.lane_graph.create_lane_graph(G, lanes_attribute=KEY_LANES_DESCRIPTION)
    snman.rebuilding._adjust_width_of_lanes(L, None, G, 'width', lanetypes={LANETYPE_CYCLING_LANE})
    snman.rebuilding._adjust_width_of_lanes(L, None, G, 'width')
    snman.street_graph.lane_graph_to_street_graph(
        G, L,
        KEY_LANES_DESCRIPTION
    )
    
    L = snman.lane_graph.create_lane_graph(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)
    snman.rebuilding._adjust_width_of_lanes(L, None, G, 'width', lanetypes={LANETYPE_CYCLING_LANE})
    snman.rebuilding._adjust_width_of_lanes(L, None, G, 'width')
    snman.street_graph.lane_graph_to_street_graph(
        G, L,
        KEY_LANES_DESCRIPTION_AFTER
    )

In [19]:
if 1:
    print('export G')
    snman.io.export_street_graph(
        G,
        os.path.join(export_path, 'G_edges.gpkg'),
        os.path.join(export_path, 'G_nodes.gpkg')
    )

if 1:
    print('export L before')
    L = snman.lane_graph.create_lane_graph(G, lanes_attribute=KEY_LANES_DESCRIPTION)
    snman.io.export_lane_geometries(
        L,
        os.path.join(export_path, 'lane_geometries_edges.gpkg'),
        os.path.join(export_path, 'lane_geometries_nodes.gpkg'),
        scaling=2
    )

if 1:
    print('export L after')
    L = snman.lane_graph.create_lane_graph(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)
    snman.io.export_lane_geometries(
        L,
        os.path.join(export_path, 'lane_geometries_rebuilt_edges.gpkg'),
        os.path.join(export_path, 'lane_geometries_rebuilt_nodes.gpkg'),
        scaling=2
    )

export G
export L before
export L after


In [20]:
if 0:
    print('Load street graph')
    G = snman.io.load_street_graph(
        os.path.join(export_path, 'G_edges.gpkg'),
        os.path.join(export_path, 'G_nodes.gpkg'),
        crs=CRS_internal
    )
    
    street_graph.organize_edge_directions(G)